# Spectrum xAI

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import sys

from src.data.dataset import DAGMDataset
from src.models.models import DenseNetClassifier
from src.models.train_model import train_model
from src.models.evaluation import evaluate_model
from src.task.gradcam import show_gradcam
from src.task.gradient_shap import show_gradient_shap
from src.task.scorecam import show_scorecam
from src.task.integrated_gradient import show_integrated_gradient
from src.task.lime import show_lime

import sys 
sys.dont_write_bytecode = True

## Data Loading

In [2]:
CLASS = 1
IMAGE_PATH = f"data/dataset/Class{CLASS}/"

In [3]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = DAGMDataset(root_dir=IMAGE_PATH, split="Train", transform=transform)
test_dataset = DAGMDataset(root_dir=IMAGE_PATH, split="Test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

## Model

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNetClassifier(pretrained=True, device=device)

criterion = torch.nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 10

### Training

In [5]:
# train_accuracies, train_losses = train_model(model, train_loader, num_epochs, criterion, optimizer, device)
# model.save_model(f"data/models/densenet121_class{CLASS}.pt")

### Evaluation

In [5]:
model.load_model(f"data/models/model_densenet121_class{CLASS}.pth")
test_accuracy = evaluate_model(model, test_loader, device)
print(f"Test Accuracy: {test_accuracy:.2f}%")

## Explanation

In [6]:
index = 1
image, label = test_dataset[index][0].unsqueeze(0), test_dataset[index][1]
print(f'True Label: {label}')

In [ ]:
show_gradcam(model.model, image, 6, device)

In [ ]:
show_gradient_shap(model.model, image, device, num_samples=100)

In [ ]:
show_scorecam(model.model, image, -2, device)

In [7]:
show_integrated_gradient(model.model, image, device, n_steps=70)